In [1]:
import keras
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout

In [2]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input,shear_range=0.1,
                                   zoom_range=0.1,horizontal_flip=True)
train_gen = train_data.flow_from_directory('../input/100-bird-species/train',
        target_size=(224, 224),batch_size=64,class_mode='categorical')
val_data= ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen= val_data.flow_from_directory(
        '../input/100-bird-species/valid',
        target_size=(224, 224),batch_size=32,class_mode='categorical')
test_data= ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen= test_data.flow_from_directory(
        '../input/100-bird-species/test',
        target_size=(224, 224),batch_size=32,class_mode='categorical')

Found 58388 images belonging to 400 classes.
Found 2000 images belonging to 400 classes.
Found 2000 images belonging to 400 classes.


# VGG16

In [3]:
base_model=keras.applications.vgg16.VGG16(include_top=False,weights="imagenet",input_shape=(224,224,3))

2022-07-24 13:09:41.904973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 13:09:42.025456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 13:09:42.026188: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 13:09:42.027449: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 0s 0us/step


In [4]:
base_model.trainable = False

In [5]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(400,activation='softmax',kernel_initializer='glorot_normal'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 2048)              51382272  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               8

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_gen,epochs=10,validation_data=val_gen,workers=10,use_multiprocessing=True)

2022-07-24 13:09:46.405701: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-07-24 13:09:55.672101: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


913/913 [==============================] - 754s 805ms/step - loss: 7.7144 - accuracy: 0.0363 - val_loss: 4.8907 - val_accuracy: 0.1645
Epoch 2/10
913/913 [==============================] - 721s 780ms/step - loss: 4.8426 - accuracy: 0.1660 - val_loss: 3.2430 - val_accuracy: 0.3920
Epoch 3/10
913/913 [==============================] - 725s 786ms/step - loss: 3.8917 - accuracy: 0.2918 - val_loss: 2.0600 - val_accuracy: 0.5550
Epoch 4/10
913/913 [==============================] - 733s 793ms/step - loss: 3.2596 - accuracy: 0.3801 - val_loss: 1.4882 - val_accuracy: 0.6645
Epoch 5/10
913/913 [==============================] - 736s 798ms/step - loss: 2.8050 - accuracy: 0.4550 - val_loss: 1.1393 - val_accuracy: 0.7345
Epoch 6/10
913/913 [==============================] - 718s 778ms/step - loss: 2.5010 - accuracy: 0.5077 - val_loss: 1.0082 - val_accuracy: 0.7570
Epoch 7/10
913/913 [==============================] - 759s 824ms/step - loss: 2.2350 - accuracy: 0.5522 - val_loss: 0.8541 - val_accura

## To get more accuracy, increase the number of epochs.